In [ ]:
#### IMPORTS ####
using ProgressMeter
# using PyPlot
using Plots
theme(:bright)

In [ ]:
function mapTo2D(points, kx_arr, ky_arr)
    kx = (points .- 1) .% num_kspace .+ 1
    ky = div.((points .- 1), num_kspace) .+ 1
    return kx_arr[kx], ky_arr[ky]
end


function isogeometricContour(kx_start, num_kspace)
    if kx_start > div(num_kspace, 2)
       kx_start =  num_kspace - kx_start
    end
    contourPoints = []
    kx_points_left = 1:kx_start
    kx_points_right = num_kspace + 1 .- kx_points_left
    
    ky_points_lowerleft = kx_start + 1 .- kx_points_left
    ky_points_upperleft = num_kspace - kx_start .+ kx_points_left
    ky_points_lowerright = kx_start - num_kspace .+ kx_points_right
    ky_points_upperright = 2 * num_kspace - kx_start + 1 .- kx_points_right
    
    push!(contourPoints, (kx_points_left .+ num_kspace .* (ky_points_lowerleft .- 1))...)
    push!(contourPoints, (kx_points_left .+ num_kspace .* (ky_points_upperleft .- 1))...)
    push!(contourPoints, (kx_points_right .+ num_kspace .* (ky_points_lowerright .- 1))...)
    push!(contourPoints, (kx_points_right .+ num_kspace .* (ky_points_upperright .- 1))...)

    return contourPoints
end


function dispersionFlattened(t, kx_arr, ky_arr)
    kx_arr_flattened = repeat(kx_arr, inner=length(ky_arr))
    ky_arr_flattened = repeat(ky_arr, outer=length(kx_arr))
    return-2*t.*(cos.(kx_arr_flattened) + cos.(ky_arr_flattened))
end


function getDOS(num_kspace, dispersionArray)
    delta_k = 2 * pi / num_kspace
    kspaceDos = 1 / delta_k
    densityOfStates = zeros(num_kspace * num_kspace)
    Threads.@threads for j in 0:num_kspace-1
        for i in 1:num_kspace
            E_xy = dispersionArray[j * num_kspace + i]
            E_xpy = dispersionArray[j * num_kspace + i % num_kspace + 1]
            E_xyp = dispersionArray[(j * num_kspace + num_kspace) % num_kspace^2 + i]
            dE_xydk = sqrt((E_xpy - E_xy)^2 / delta_k^2
                           + (E_xyp - E_xy)^2 / delta_k^2
                           )
            densityOfStates[j * num_kspace + i] = kspaceDos / dE_xydk
        end
    end
    replace!(densityOfStates, Inf=>maximum(densityOfStates[densityOfStates .≠ Inf]))
    return densityOfStates
end


function getIsoEnergeticContour(dispersionArray, num_kspace, energy)
    contourPoints = [[] for j in 1:num_kspace]
    Threads.@threads for j in 0:num_kspace-1
        energyDiffArr = dispersionArray[j * num_kspace + 1: (j + 1) * num_kspace] .- energy
        for i in 1:num_kspace
            if energyDiffArr[i] * energyDiffArr[i % num_kspace + 1] <= 0
                push!(contourPoints[j+1], j * num_kspace + i)
            end
        end
    end
    return collect(Iterators.flatten(contourPoints))
end


function getRenorm(args)
    omegaArray, k1, Ek1, k2_arr, Ek2_arr, cutoffPoints, energyCutoff, densityOfStates, kondoJArray, flags_k1_arr, deltaD = args
    renormArr = zeros(size(k2_arr))
    J_k1 = kondoJArray[k1,cutoffPoints]
    for (index, (k2, Ek2)) in collect(enumerate(zip(k2_arr, Ek2_arr)))
        J_k2 = kondoJArray[k2,cutoffPoints]
        if flags_k1_arr[k2] == 0
            continue
        end
        denominators_p = omegaArray .- energyCutoff ./ 2 .- (Ek1 / 2 + Ek2 / 2) .+ (J_k1 + J_k2) ./ 4
        denominators_h = omegaArray .- energyCutoff ./ 2 .+ (Ek1 / 2 + Ek2 / 2) .+ (J_k1 + J_k2) ./ 4
        if ! (all(<(0), denominators_p)  && all(<(0), denominators_h))
            flags_k1_arr[k2] = 0
        else
            renormArr[index] = -0.5 * sum(J_k1 .* J_k2 .* densityOfStates[cutoffPoints] 
                * deltaD .* (1 ./ denominators_p + 1 ./ denominators_h))
        end
    end
    return k1, flags_k1_arr, renormArr
end


function main(num_kspace::Int64, t, J_init)
    kx_arr = range(-pi, stop=pi, length=num_kspace)
    ky_arr = copy(kx_arr)
    dispersionArray = dispersionFlattened(t, kx_arr, ky_arr)
    densityOfStates = getDOS(num_kspace, dispersionArray)
    innerIndicesArr = collect(1:length(dispersionArray))
    
    deltaD = maximum(dispersionArray) / (num_kspace - 1)
    kondoJArray = Array{Float64}(undef, num_kspace^2, num_kspace^2, div(num_kspace-1,2)+1)
    kondoJArray[:,:,1] .= J_init
    flags = fill(1, num_kspace^2, num_kspace^2)
    @showprogress for stepIndex in 1:div(num_kspace-1,2)
        kondoJArray[:,:,stepIndex+1] = kondoJArray[:,:,stepIndex]
        # cutoffPoints = getContour(dispersionArray, num_kspace, energyCutoff)
        cutoffPoints = isogeometricContour(stepIndex, num_kspace)
        energyCutoff = dispersionArray[cutoffPoints]
        flags[cutoffPoints,:] .= 0
        flags[:,cutoffPoints] .= 0
        if all(==(0), flags)
            kondoJArray[:,:,stepIndex+1:end] .= kondoJArray[:,:,stepIndex]
            break
        end
        innerIndicesArr = setdiff(innerIndicesArr, cutoffPoints)
        omegaArray = -energyCutoff ./ 2
        innerEnergiesArr = dispersionArray[innerIndicesArr]
        Threads.@threads for (k1, Ek1) in collect(zip(innerIndicesArr, innerEnergiesArr))
            args = (omegaArray, k1, Ek1, innerIndicesArr, innerEnergiesArr, 
                cutoffPoints, energyCutoff, densityOfStates, 
                kondoJArray[:,:,stepIndex], flags[k1,:], deltaD)
            k1, flags_k1_arr, renormArr = getRenorm(args)
            flags[k1,:] = flags_k1_arr
            kondoJArray[k1,innerIndicesArr,stepIndex+1] += renormArr
            kondoJArray[k1,innerIndicesArr,stepIndex+1][kondoJArray[k1,innerIndicesArr,stepIndex] .* kondoJArray[k1,innerIndicesArr,stepIndex+1] .< 0] .= 0
        end
    end
    # println(kondoJArray[10,15,:])
    return kondoJArray, dispersionArray
end

In [ ]:
p = plot(xlims = (-1.1*pi, 1.1*pi), ylims = (-1.1*pi, 1.1*pi), size=(500, 500), legend=false, thickness_scaling=1.5)
num_kspace = 31
kx_arr = range(-pi, stop=pi, length=num_kspace)
ky_arr = copy(kx_arr)
dispersionArray = dispersionFlattened(t, kx_arr, ky_arr)
t = 1.0
@time kondoJArray, dispersionArray = main(num_kspace, t, 0.04);
kFpoints = getIsoEnergeticContour(dispersionArray, num_kspace, 0)
kFNode = 736
kFAntiNode = 916
kx, ky = mapTo2D([kFNode, kFAntiNode], kx_arr, ky_arr)
scatter!(p, kx, ky)
k2points = getIsoEnergeticContour(dispersionArray, num_kspace, 2 * t)
k2Node = 771
k2AntiNode = 953
kx, ky = mapTo2D([k2Node, k2AntiNode], kx_arr, ky_arr)
scatter!(p, kx, ky)
display(p)
# kx, ky = mapTo2D(kFpoints, kx_arr, ky_arr)
# scatter!(p, kx, ky)
p = plot(line_width=3, thickness_scaling=1.5)
for (p1, p2) in zip([kFNode, kFAntiNode], [k2Node, k2AntiNode])
    plot!(kondoJArray[p1,p2,:], label=(p1, p2))
end
display(p)

In [ ]:
num_kspace = 21
p = plot(xlims = (-1.1*pi, 1.1*pi), ylims = (-1.1*pi, 1.1*pi), size=(500, 500), legend=false, thickness_scaling=1.5)
kx_arr = range(-pi, stop=pi, length=num_kspace)
ky_arr = copy(kx_arr)
points = isogeometricContour(4, num_kspace)
for point in points
    # println(point)
    x = (point - 1) % num_kspace + 1
    y = div((point - 1), num_kspace) + 1
    # println(point, " ", x, " ", y)
    scatter!([kx_arr[x]], [kx_arr[y]])
end
display(p)

In [ ]:
num_kspace = 21
kx_arr = range(-pi, stop=pi, length=num_kspace)
ky_arr = copy(kx_arr)
for (kx, ky) in Iterators.product(kx_arr, ky_arr)
    gamma = 